In [25]:
import numpy as np
import pandas as pd
import transformers
import torch
import json
import torch.nn as nn
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from transformers import RobertaTokenizer, RobertaModel
from sklearn.metrics import classification_report
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef
from transformers import RobertaTokenizer, RobertaModel

In [26]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

# Datos originales

In [27]:
df = pd.read_csv('dataset_en_train_completed.csv')
df=df.iloc[:4000,:]
df.category.value_counts()

category
CRITICAL      2621
CONSPIRACY    1379
Name: count, dtype: int64

# Datos balanceados (Aumento de Datos)

In [37]:
df= pd.read_csv('dataset_en_train_completed.csv')
df= df.iloc[:4000]

df_augmented= pd.read_csv('dataset_en_train_completed.csv')
df_augmented= df_augmented.iloc[4000:]
#df_augmented

In [38]:
original_counts = df['category'].value_counts()
df_conspiracy = df[df['category'] == 'CONSPIRACY']
df_conspiracy_augmented = df_augmented[df_augmented['category'] == 'CONSPIRACY']
critical_count = original_counts.get('CRITICAL', 0) 
conspiracy_count = original_counts.get('CONSPIRACY', 0)


In [39]:
df_conspiracy = df_augmented[df_augmented['category'] == 'CONSPIRACY']

# Seleccionar 1242 filas aleatorias para balancear el dataset 
df_conspiracy_sampled = df_conspiracy.sample(n=1242, random_state=42)
df_combined = pd.concat([df, df_conspiracy_sampled])

df_combined.reset_index(drop=True, inplace=True)
df=df_combined.copy()
df

,id,text,category,annotations,spacy_tokens,matched_words,top_3_emotions,max_emotion,top_morals,max_moral
0,5206,this is massive australian senator malcolm rob...,CONSPIRACY,[{'span_text': 'Australian Senator Malcolm Rob...,WyJUSElTIiwgIklTIiwgIk1BU1NJVkUiLCAiQXVzdHJhbG...,"['senator', 'politician', 'are', 'roberts', 'm...","['inspiration', 'amusement', 'anger']",inspiration,"{'harm': 0.0, 'cheating': 0.0, 'betrayal': 0.0...",no moral
1,1387,i m deeply concerned that the push to vaccinat...,CRITICAL,[{'span_text': 'I ’m deeply concerned that the...,WyJcdTIwMWMiLCAiSSIsICJcdTIwMTltIiwgImRlZXBseS...,"['concerned', 'consequences', 'shots', 'dystop...","['inspiration', 'amusement', 'fear']",inspiration,"{'harm': 0.0, 'cheating': 0.0, 'betrayal': 0.0...",no moral
2,13116,they wanted to know your vaccination status an...,CRITICAL,"[{'span_text': 'someone who died suddenly', 'c...",WyIyMDIxIiwgIjoiLCAiVGhleSIsICJ3YW50ZWQiLCAidG...,"['see', 'be', 'someone', 'nt', 'allowed', 'res...","['inspiration', 'indifference', 'amusement']",inspiration,"{'harm': 0.0, 'cheating': 0.0, 'betrayal': 0.0...",authority
3,11439,anthony fauci once again defended brutal chine...,CRITICAL,"[{'span_text': 'brutal Chinese lockdowns', 'ca...",WyJBbnRob255IiwgIkZhdWNpIiwgIm9uY2UiLCAiYWdhaW...,"['news', 'defends', 'forcefully', 'chinese', '...","['anger', 'annoyance', 'inspiration']",anger,"{'harm': 2.33333333333, 'cheating': 0.0, 'betr...",harm
4,98,proof has emerged showing that death from wuha...,CRITICAL,[{'span_text': 'death from Wuhan coronavirus (...,WyJQcm9vZiIsICJoYXMiLCAiZW1lcmdlZCIsICJzaG93aW...,"['death', 'alive', 'due', 'infinite', 'breakin...","['inspiration', 'amusement', 'fear']",inspiration,"{'harm': 2.1666666666666696, 'cheating': 0.0, ...",harm
...,...,...,...,...,...,...,...,...,...,...
5237,E0S3P,. zickute . com video ubxufaftemjq uk police c...,CONSPIRACY,NaN,NaN,"['death', 'in', 'country', 'org', 'stop', 'dam...","['inspiration', 'amusement', 'anger']",inspiration,"{'harm': 3.0, 'cheating': 0.0, 'betrayal': 4.5...",harm
5238,ffdyY,get a claim in for medical battery. nbcnews. c...,CONSPIRACY,NaN,NaN,"['death', 'in', 'claim', 'science', 'vast', 'o...","['inspiration', 'amusement', 'anger']",inspiration,"{'harm': 0.0, 'cheating': 0.0, 'loyalty': 8.0,...",loyalty
5239,BJ3ns,We warn against ... vaccine mandates health pa...,CONSPIRACY,NaN,NaN,"['system', 'abolition', 'climate', 'digital', ...","['inspiration', 'amusement', 'fear']",inspiration,"{'harm': 0.0, 'cheating': 0.0, 'betrayal': 0.0...",degradation
5240,qFOON,Vaccination center in leaks Crime against huma...,CONSPIRACY,NaN,NaN,"['crimes', 'leaks', 'humanity', 'medical', 'in...","['anger', 'annoyance', 'inspiration']",anger,"{'harm': 0.0, 'cheating': 0.0, 'betrayal': 0.0...",no moral


# Transformar etiqueta de categórica a numérica

Critical = 1
Conspirancy = 0

In [40]:
df['class'] = df['category'].apply(lambda x: 1 if x == 'CRITICAL' else 0)

new_df = df[['text', 'class','top_3_emotions']].copy()
new_df

,text,class,top_3_emotions
0,this is massive australian senator malcolm rob...,0,"['inspiration', 'amusement', 'anger']"
1,i m deeply concerned that the push to vaccinat...,1,"['inspiration', 'amusement', 'fear']"
2,they wanted to know your vaccination status an...,1,"['inspiration', 'indifference', 'amusement']"
3,anthony fauci once again defended brutal chine...,1,"['anger', 'annoyance', 'inspiration']"
4,proof has emerged showing that death from wuha...,1,"['inspiration', 'amusement', 'fear']"
...,...,...,...
5237,. zickute . com video ubxufaftemjq uk police c...,0,"['inspiration', 'amusement', 'anger']"
5238,get a claim in for medical battery. nbcnews. c...,0,"['inspiration', 'amusement', 'anger']"
5239,We warn against ... vaccine mandates health pa...,0,"['inspiration', 'amusement', 'fear']"
5240,Vaccination center in leaks Crime against huma...,0,"['anger', 'annoyance', 'inspiration']"


# Parámetros



In [41]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 3
LEARNING_RATE = 2e-5
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        row = self.dataframe.iloc[index]

        text = row['text']
        important_emotions = row['top_3_emotions'] 

        # Tokenizamos el texto original
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True
        )
        
        ids = torch.tensor(inputs['input_ids'])
        mask = torch.tensor(inputs['attention_mask'])

        # Tokenizamos las emociones
        emotion_inputs = self.tokenizer.encode(important_emotions, add_special_tokens=True)
        emotion_ids = torch.tensor(emotion_inputs)

        targets = torch.tensor(row['class'])  
        return {
            'ids': ids,
            'mask': mask,
            'important_word_ids': emotion_ids,  
            'targets': targets
        }


# MODELO

In [ ]:
class RoBERTaClass(torch.nn.Module):
    def __init__(self):
        super(RoBERTaClass, self).__init__()
        # Cargar el modelo preentrenado RoBERTa
        self.l1 = transformers.RobertaModel.from_pretrained('roberta-base', output_attentions=True)
        self.multihead_attention = nn.MultiheadAttention(embed_dim=768, num_heads=8, dropout=0.3, batch_first=True)

        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 1)  # Clasificación binaria

    def forward(self, ids, mask, important_word_ids=None, important_emotions=None):
        # Paso 1: Obtener embeddings del modelo para el texto original
        outputs = self.l1(input_ids=ids, attention_mask=mask)
        
        # Los embeddings de las palabras del texto original (sin alteraciones) están en `outputs.last_hidden_state`
        original_embeddings = outputs.last_hidden_state  # Shape: [batch_size, seq_len, embedding_dim]

        # Paso 2: Obtener embeddings de las palabras de las emociones (que pasarán por el modelo también)
        if important_emotions is not None:
            # Tokenizamos las palabras emocionales y las pasamos por el modelo RoBERTa para obtener sus embeddings
            emotion_inputs = self.l1.encoder.embeddings(important_emotions)  # Embeddings de las palabras emocionales
            # `important_emotions` debe ser un tensor que contiene las IDs de las palabras de emoción

            # Concatenar los embeddings originales con los de las emociones
            # Asumimos que 'important_emotions' tiene un tamaño compatible para la concatenación
            concatenated_embeddings = torch.cat((original_embeddings, emotion_inputs), dim=1)

        else:
            # Si no se pasan las emociones, simplemente usamos los embeddings originales
            concatenated_embeddings = original_embeddings

        # Paso 3: Pasar los embeddings concatenados a través de la red
        output_2 = self.l2(concatenated_embeddings.mean(dim=1))  # Promedio de las representaciones concatenadas
        output = self.l3(output_2).squeeze(1)  # Clasificación binaria
        return output


model = RoBERTaClass().to(device)

def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)


# Fine Tuning (Cross Validation)

In [ ]:
def collate_fn(batch):
    max_important_word_ids_len = max(len(sample['important_word_ids']) for sample in batch)

    for sample in batch:
        sample_len = len(sample['important_word_ids'])
        if sample_len < max_important_word_ids_len:
            sample['important_word_ids'] = torch.cat([sample['important_word_ids'], 
                                                      torch.tensor([-1] * (max_important_word_ids_len - sample_len))])
        sample['important_word_ids'] = torch.tensor(sample['important_word_ids'], dtype=torch.long)

    return {
        'ids': torch.stack([sample['ids'] for sample in batch]),
        'mask': torch.stack([sample['mask'] for sample in batch]),
        'important_word_ids': torch.stack([sample['important_word_ids'] for sample in batch]),
        'targets': torch.tensor([sample['targets'] for sample in batch], dtype=torch.float)
    }



def train(epoch, model, train_loader, optimizer, loss_fn, device):
    model.train()
    for _, data in enumerate(train_loader):
        ids = data['ids'].to(device, dtype=torch.long)
        mask = data['mask'].to(device, dtype=torch.long)
        targets = data['targets'].to(device, dtype=torch.float)
        important_word_ids = data['important_word_ids'].to(device, dtype=torch.long)

        optimizer.zero_grad()
        outputs = model(ids, mask, important_word_ids=important_word_ids)
        loss = loss_fn(outputs, targets)

        if _ % 5000 == 0:
            print(f'Epoch: {epoch}, Loss: {loss.item()}')

        loss.backward()
        optimizer.step()


def validation(model, data_loader, device):
    model.eval()
    fin_targets = []
    fin_outputs = []

    with torch.no_grad():
        for data in data_loader:
            ids = data['ids'].to(device)
            mask = data['mask'].to(device)
            targets = data['targets'].to(device)
            important_word_ids = data['important_word_ids'].to(device)

            outputs = model(ids, mask, important_word_ids=important_word_ids)
            outputs = torch.sigmoid(outputs)

            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(outputs.cpu().detach().numpy().tolist())

    return fin_outputs, fin_targets


# Evaluación de métricas
def evaluate_metrics(outputs, targets):
    # Convertir las salidas a 0 o 1 (clases predichas) basadas en el umbral 0.5
    outputs = [1 if x > 0.5 else 0 for x in outputs]

    # Calcular las métricas
    accuracy = accuracy_score(targets, outputs)
    precision, recall, f1, _ = precision_recall_fscore_support(targets, outputs, average='binary')
    mcc = matthews_corrcoef(targets, outputs)  # Calcular MCC

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'mcc': mcc
    }

# Función de validación cruzada
def cross_validate_model(model, dataframe, tokenizer, title, epochs=3, batch_size=16, k_folds=5, device='cuda'):
    # KFold
    kf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)
    metrics_list = []
    for fold, (train_idx, val_idx) in enumerate(kf.split(dataframe, dataframe['class'])):
        print(f"\nFold {fold + 1}/{k_folds}")
        train_df = dataframe.iloc[train_idx].reset_index(drop=True)
        val_df = dataframe.iloc[val_idx].reset_index(drop=True)

        # Crear datasets
        train_set = CustomDataset(train_df, tokenizer, max_len=256)
        val_set = CustomDataset(val_df, tokenizer, max_len=256)

        # Crear dataloaders
        train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
        val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

        # Inicializar el optimizador y la función de pérdida
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
        loss_fn = torch.nn.BCEWithLogitsLoss()

        # Entrenar el modelo
        for epoch in range(epochs):
            train(epoch, model, train_loader, optimizer, loss_fn, device)

        # Validar el modelo
        outputs, targets = validation(model, val_loader, device)

        # Evaluar las métricas
        fold_metrics = evaluate_metrics(outputs, targets)
        metrics_list.append(fold_metrics)

    # Guardar métricas en un DataFrame
    metrics_df = pd.DataFrame(metrics_list)
    metrics_df.to_csv(f'metrics_{title}.csv', index=False)

    print('Cross-validation complete')
cross_validate_model(model, new_df, tokenizer, 'roberta_final',epochs=3, batch_size=TRAIN_BATCH_SIZE, k_folds=5)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Fold 1/5


/tmp/ipykernel_128717/1513134092.py:99: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample['important_word_ids'] = torch.tensor(sample['important_word_ids'], dtype=torch.long)


Epoch: 0, Loss: 0.7399255037307739
Epoch: 1, Loss: 0.282997190952301
Epoch: 2, Loss: 0.05724325031042099

Fold 2/5


/tmp/ipykernel_128717/1513134092.py:99: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample['important_word_ids'] = torch.tensor(sample['important_word_ids'], dtype=torch.long)


...

Epoch: 0, Loss: 0.0061414288356900215
Epoch: 1, Loss: 0.027896394953131676
Epoch: 2, Loss: 0.010822286829352379

Fold 4/5


/tmp/ipykernel_128717/1513134092.py:99: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample['important_word_ids'] = torch.tensor(sample['important_word_ids'], dtype=torch.long)


Epoch: 0, Loss: 0.21250909566879272
Epoch: 1, Loss: 0.00196507154032588
Epoch: 2, Loss: 0.0027468749321997166

Fold 5/5


/tmp/ipykernel_128717/1513134092.py:99: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample['important_word_ids'] = torch.tensor(sample['important_word_ids'], dtype=torch.long)


Epoch: 0, Loss: 0.0009135260479524732
Epoch: 1, Loss: 0.0006676175398752093
Epoch: 2, Loss: 0.0006014542886987329
Cross-validation complete


# Test

In [47]:
test_df = pd.read_csv("dataset_en_test_cleaned.csv")
test_df['class'] = test_df['category'].apply(lambda x: 1 if x == 'CRITICAL' else 0)
test_df = test_df[['text', 'class']].copy()
test_df

,text,class
0,elon now confirming what we ve been suspecting...,0
1,keeping the pressure on the police to uphold t...,0
2,safe effective the greatest lie ever told . th...,1
3,cdc report admits . million people in the usa ...,1
4,how to use health to acquire totalitarian cont...,0
...,...,...
995,john d. rockefeller wiped out natural cures to...,0
996,fact check biden white house falsely accuses d...,1
997,w onset acral hand lesions following mrna vacc...,1
998,we will fire unvaccinated workers cohen hadad ...,1


In [48]:
model.eval()
def predict(text, model, tokenizer):
    inputs = tokenizer.encode_plus(
        text,
        None,
        add_special_tokens=True,
        max_length=MAX_LEN,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )

    ids = inputs['input_ids'].to(device)
    mask = inputs['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(ids, mask)
        probabilities = torch.sigmoid(outputs).cpu().numpy()
        prediction = 1 if probabilities[0] >= 0.5 else 0

    return prediction, probabilities[0]


def test_and_evaluate(model, tokenizer, test_df, filename="test_results.json"):
    predictions = []
    probabilities = []

    for index, row in test_df.iterrows():
        text = row['text']
        prediction, probability = predict(text, model, tokenizer)  
        predictions.append(prediction)
        probabilities.append(probability)

    test_df['predictions'] = predictions
    test_df['probabilities'] = probabilities

    mcc = matthews_corrcoef(test_df['class'], test_df['predictions'])
    results = classification_report(
        test_df['class'],
        test_df['predictions'],
        target_names=['CONSPIRANCY', 'CRITICAL'], 
        digits=5,
        output_dict=True
    )

    output_data = {
        "mcc": mcc,
        "classification_report": results
    }

    with open(filename, "w") as f:
        json.dump(output_data, f, indent=4)

    print(f"Test results saved to {filename}")
    print(f"Test MCC = {mcc}")

# Example usage:
test_and_evaluate(model, tokenizer, test_df, filename="roberta_final.json")

Test results saved to roberta_final.json
Test MCC = 0.8046594706827834
